<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/operanalyst/teoriepruzkumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teorie průzkumu

## Úvaha

Představte si, že máte prostor rozdělený na dvě části. Zvolíte si jednu část, kterou zkontrolujete. Protivník si taktéž zvolí část, kde bude přítomen. V případě, kdy zvolíte stejný prostor, vyhráváte, v opačném případě prohráváte.



$$max\{v:A^Tx^B- \begin{pmatrix}v \\
v \\
\vdots \\
v
\end{pmatrix} \ge 0, \sum_{i=1}^nx_i^B=1,x^B \ge 0, v \in R\}$$

## Pravděpodobnost setkání

https://www.mathpages.com/home/kmath124/kmath124.htm

Mějme dvě náhodné proměnné s rovnoměrným rozdělením pravděpodobností na intervalu $<0;1>$. Jaká je pravděpodobnost, že jejich hodnoty jsou od sebe vzdálené maximálně $q$? 

$$P_2=q$$

Zobecněný vztah pro $n$ náhodných proměnných je

$$P_n=nq^{n-1}-(n-1)q^n$$



In [ ]:
Teorie průzkumu